## Load data

In [1]:
import sys
 
sys.path.insert(0, "../")

In [2]:
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd

data = pd.read_csv(r"../extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/sbc/feature_variation/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Training: 
Assessable data are 528101 cases and 1015074 CBCs
Control data are 527038 cases and 1013548 CBCs
Sepsis data are 1488 cases and 1526 CBCs
$$$$$$$$$$$$$$$$$$$$
Testing: 


/home/dwalke/git/sbc/feature_variation/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 365794, Sepsis: 490
Assessable data are 180494 cases and 366284 CBCs
Control data are 180157 cases and 365794 CBCs
Sepsis data are 472 cases and 490 CBCs


/home/dwalke/git/sbc/feature_variation/../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']


Controls: 437629, Sepsis: 448
Assessable data are 157922 cases and 438077 CBCs
Control data are 180157 cases and 437629 CBCs
Sepsis data are 438 cases and 448 CBCs


In [3]:
import torch

y_train = torch.tensor(data_analysis.get_y_train(), dtype=torch.long)
X_train = torch.tensor(data_analysis.get_X_train(), dtype=torch.float)

## Define model

In [4]:
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv,GATv2Conv, GINConv, global_add_pool
from torch.nn import Linear
import torch
from dataAnalysis.Constants import FEATURES
from torch.nn import Linear, ReLU, Sequential
from torch.nn import BatchNorm1d as BatchNorm
from torch_geometric.loader import NeighborLoader

class GraphNeuralNetwork(torch.nn.Module):
    def __init__(self, hidden_dim = 128, out_channels = 1):
        super(GraphNeuralNetwork, self).__init__()
        input_dim = len(FEATURES)      
        
        HEADS = 5
        
        conv_1= GATConv(input_dim, hidden_dim,heads=HEADS, add_self_loops = False)
        conv_end = GATConv((-1,-1), out_channels,add_self_loops = False)
        
        self.conv_1 = conv_1
        self.conv_end = conv_end
        

    def forward(self, graph):
        x, edge_index = graph.x, graph.edge_index
        x = x.type(torch.float)
        x = self.conv_1(x, edge_index)
        x = F.normalize(x, p=2., dim=-1)
        x = torch.relu(x)
        x = self.conv_end(x, edge_index)
        return x
            
    def predict_proba(self, graph):
        y_pred_proba_all = torch.tensor([])
        loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size = 50_000)
        with torch.inference_mode():
            self.eval()
            for i, batch in enumerate(loader):
                logits = self.forward(batch)
                scores = torch.sigmoid(torch.squeeze(logits))
                scores = torch.unsqueeze(scores, 0)
                proba_predict = torch.concat((1- scores, scores), dim = 0)
                transp_proba_pred = torch.transpose(proba_predict, 0, 1)
                y_pred_proba_all = torch.concat((y_pred_proba_all, transp_proba_pred), dim = 0)
        return y_pred_proba_all
            
    def predict(self, graph):
        return torch.round(self.predict_proba(graph)[:, 1])

## Load trained model

In [5]:
model = GraphNeuralNetwork().cpu() #.to(device)
model.load_state_dict(torch.load("../models/directed_gat_wo_pos.pt", map_location=torch.device('cpu')))
model.eval()

GraphNeuralNetwork(
  (conv_1): GATConv(7, 128, heads=5)
  (conv_end): GATConv((-1, -1), 1, heads=1)
)

## Create graph from synthetic dataset

In [6]:
from torch_geometric.nn import knn_graph
from dataAnalysis.FeatureImportance import FeatureImportance

def normalize(tensor):
    if not torch.is_tensor(tensor):
        tensor = torch.from_numpy(tensor).type(torch.float)
    mean = torch.mean(tensor, dim = 0)
    std = torch.std(tensor, dim = 0)
    mean_diff = tensor - mean
    return mean_diff / std


feature_importance = FeatureImportance(X_train)
X_all_fv = normalize(feature_importance.X_all_fv)
node_indices = torch.arange(X_all_fv.shape[0]).type(torch.long)
edge_index = node_indices.repeat(2 ,1)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from dataAnalysis.Constants import FEATURES

def getPositionEncoding(seq_len, d = len(FEATURES), n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P

getPositionEncoding(1)

array([[0., 1., 0., 1., 0., 1., 0.]])

In [8]:
X_all_fv

tensor([[-1.8624, -1.0000, -2.8744,  ..., -2.5111, -2.2260, -0.6495],
        [-1.8624, -1.0000, -2.8744,  ..., -2.5111, -2.2260, -0.5297],
        [-1.8624, -1.0000, -2.8744,  ..., -2.5111, -2.2260, -0.4285],
        ...,
        [ 2.2867,  1.0000,  2.6466,  ...,  3.1966,  3.7011,  0.0342],
        [ 2.2867,  1.0000,  2.6466,  ...,  3.1966,  3.7011,  0.1485],
        [ 2.2867,  1.0000,  2.6466,  ...,  3.1966,  3.7011,  4.2864]])

In [9]:
X_all_fv + getPositionEncoding(1).repeat(X_all_fv.shape[0], 0)

tensor([[-1.8624,  0.0000, -2.8744,  ..., -2.5111, -1.2260, -0.6495],
        [-1.8624,  0.0000, -2.8744,  ..., -2.5111, -1.2260, -0.5297],
        [-1.8624,  0.0000, -2.8744,  ..., -2.5111, -1.2260, -0.4285],
        ...,
        [ 2.2867,  2.0000,  2.6466,  ...,  3.1966,  4.7011,  0.0342],
        [ 2.2867,  2.0000,  2.6466,  ...,  3.1966,  4.7011,  0.1485],
        [ 2.2867,  2.0000,  2.6466,  ...,  3.1966,  4.7011,  4.2864]],
       dtype=torch.float64)

In [10]:
from torch_geometric.data import Data

# graph = Data(x= X_all_fv+getPositionEncoding(1).repeat(X_all_fv.shape[0], 0),  edge_index = edge_index.type(torch.long))
graph = Data(x= X_all_fv,  edge_index = edge_index.type(torch.long))

## Feature importance

In [11]:
feature_importance.set_model_input([graph])

In [12]:
import sys
!{sys.executable} -m pip install torch-sparse

  Using cached torch_sparse-0.6.17.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [53 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/eye.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/spmm.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/narrow.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/convert.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/spadd.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/index_select.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/reduce

In [13]:
feature_importance.plot_feature_importance(model, title= "Directed GAT", write = False)

/home/dwalke/miniconda3/envs/venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:50: UserWarning: Using '{self.__class__.__name__}' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn("Using '{self.__class__.__name__}' without a "


ImportError: 'NeighborSampler' requires either 'pyg-lib' or 'torch-sparse'

In [8]:
import torch
from torch_geometric.loader import NeighborLoader
from torch_geometric.loader import NeighborSampler

from torch_geometric.data import Data

graph = Data(x = torch.Tensor([[0,1],[0,1]]), edge_index=torch.Tensor([[0,1],[0,1]]).type(torch.long), y = torch.Tensor([0,1]))
loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size = 50_000)
with torch.inference_mode():
    for i, batch in enumerate(loader):
        print(batch)
        break

/home/dwalke/miniconda3/envs/venv/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py:50: UserWarning: Using '{self.__class__.__name__}' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn("Using '{self.__class__.__name__}' without a "


ImportError: 'NeighborSampler' requires either 'pyg-lib' or 'torch-sparse'